# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys

sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [2]:
output = open("./some_test_data.txt", 'w')

In [3]:
output.write('123')

3

In [4]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [5]:
import datetime
datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2023-12-31 14:59:36'

In [6]:
import datetime

original_write = sys.stdout.write

def my_write(string_text):
    tmstmp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    prefix = f'[{tmstmp}]: '
    
    if string_text == '\n':
        res = original_write(string_text)
    else:
        res = original_write(prefix + string_text)
    return res
    
sys.stdout.write = my_write

In [7]:
print('1, 2, 3')

[2023-12-31 14:59:37]: 1, 2, 3


In [8]:
sys.stdout.write = original_write

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [9]:
import sys
import datetime
from functools import wraps

original_write = sys.stdout.write

def timed_output(function):
    @wraps(function)
    def wrapper(*args, **kwargs):
        sys.stdout.write = my_write
        result = function(*args, **kwargs)
        sys.stdout.write = original_write
        return result

    def my_write(string_text):
        tmstmp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        prefix = f'[{tmstmp}]: '

        if string_text == '\n':
            res = original_write(string_text)
        else:
            res = original_write(prefix + string_text)
        return res


    return wrapper


In [10]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [11]:
print_greeting("Nikita")

[2023-12-31 14:59:37]: Hello, Nikita!


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [12]:
def redirect_output(filepath: str, mode='w'):
    original_write = sys.stdout
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            file_write = open(filepath, mode)
            sys.stdout = file_write
            rt = func(*args, **kwargs)
            file_write.close()
            sys.stdout = original_write
            return rt
        return wrapper
    return decorator

In [13]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [14]:
calculate()

In [15]:
%cat function_output.txt

/bin/bash: /home/yaroslav/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 
